# Import Libraries

In [1]:
%%time
#Import Libraries
import json
import spotipy
import spotipy.util as util
import pandas as pd
import numpy as np
import time
import random
import csv
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from spotipy.oauth2 import SpotifyClientCredentials

Wall time: 3.81 s


# Authentication

In [2]:
%%time
#assign parameters for authorization
client_id = '122378e2606d48a7bb61507f17d7e3e9'
client_secret = 'adeed656de8642cc895bb6e09e6b1a1e'
username = 'jofussaures'
scope = 'user-read-playback-state user-library-read user-modify-playback-state'
redirect_uri = 'https://localhost:8080/callback/'

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

#initiates spotipy with token, authorized by user
sp = spotipy.Spotify(auth = token)

Wall time: 218 ms


# If user did not do the algorithm
### if user did do algorithm, scroll down

In [ ]:
%%time
uri = []
number_of_tracks = 50

#Gets user's Saved Tracks (Liked Songs)

for i in range(0,10000,50):
    all_tracks = sp.current_user_saved_tracks(number_of_tracks, i)
    for item in all_tracks['items']:
        track = item['track']
        uri.append(track['uri'])

In [ ]:
%%time
tracks = []

#Method to get audio features

def getTrackFeatures(uri):
    meta = sp.track(uri)
    features = sp.audio_features(uri)
    
  # meta
    name = meta['name']
    artists = meta['album']['artists'][0]['name']
    year = meta['album']['release_date'][:4]
    trackUri = meta['uri']
    
  # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    key = features[0]['key']
    liveness = features[0]['liveness']
    mode = features[0]['mode']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    valence = features[0]['valence']

    track = [name, artists, year, acousticness, danceability, energy, instrumentalness, key, 
             liveness, mode, loudness, speechiness, tempo, valence, uri]
    return track

#Puts data into DataFrame
for i in range(len(uri)):
    time.sleep(.005)
    track = getTrackFeatures(uri[i])
    tracks.append(track)
    
df = pd.DataFrame(tracks, columns = ['name', 'artists', 'year', 'acousticness', 'danceability', 'energy', 'instrumentalness',
                                     'key', 'liveness', 'mode','loudness', 'speechiness', 'tempo', 'valence', 'uri'])

#Writes DataFrame to CSV file
df.to_csv("spotify.csv", sep = ',')

In [ ]:
%%time
#Import datasheet with pandas
df = pd.read_csv('spotify.csv')

In [ ]:
%%time
#dfw = dataframe w/o song name, artists, and URI
da = df['artists']
dn = df['name']
du = df['uri']
dfw=df.drop(columns = ['name','artists', 'uri'])

In [ ]:
%%time
#features = the audio features used for algorithm
features = dfw[['acousticness','danceability','energy','instrumentalness','loudness','speechiness','tempo','valence']]

In [ ]:
%%time
#Machine learning Algorithm
km = KMeans(n_clusters = 4)
km.fit(dfw)
cluster = km.labels_

newdf=dfw.assign(Cluster=cluster)
grouped=newdf.groupby(["Cluster"])
grouped.agg({'acousticness' : 'mean','danceability' : 'mean','energy' : 'mean','instrumentalness' : 'mean','loudness' : 'mean',
             'speechiness' : 'mean','tempo' : 'mean','valence' : 'mean',})

newdf_melt=pd.melt(newdf.reset_index(), id_vars =['Cluster'],value_vars=['acousticness','danceability','energy',
                                                                         'instrumentalness','loudness','speechiness','tempo',
                                                                         'valence'],var_name='Metric',value_name='Value')

In [ ]:
%%time
#Assigns clusters to a DataFrame
c0 = newdf[newdf['Cluster']==0].reindex()
c1 = newdf[newdf['Cluster']==1].reindex()
c2 = newdf[newdf['Cluster']==2].reindex()
c3 = newdf[newdf['Cluster']==3].reindex()

In [ ]:
%%time
#Adds song names, artists, and URIs back into clusters (at end of DataFrame)
#c#a = cluster# w/ artist
#c#an = cluster# w/ artist and name
#c#f = cluster# w/ full data
c0a = c0.assign(artists=df['artists'])
c0an = c0a.assign(name=df['name'])
c0f = c0an.assign(uri=df['uri'])

c1a = c1.assign(artists=df['artists'])
c1an = c1a.assign(name=df['name'])
c1f = c1an.assign(uri=df['uri'])

c2a = c2.assign(artists=df['artists'])
c2an = c2a.assign(name=df['name'])
c2f = c2an.assign(uri=df['uri'])

c3a = c3.assign(artists=df['artists'])
c3an = c3a.assign(name=df['name'])
c3f = c3an.assign(uri=df['uri'])

In [ ]:
%%time
#Puts artists and name at beginning of DataFrame
neworder = ['name','artists','year','acousticness','danceability','energy','instrumentalness','key','liveness',
            'loudness','mode','speechiness','tempo','valence','Cluster', 'uri']
c0f = c0f.reindex(columns = neworder)
c1f = c1f.reindex(columns = neworder)
c2f = c2f.reindex(columns = neworder)
c3f = c3f.reindex(columns = neworder)

In [ ]:
%%time
#Combines the four DataFrames into one
fourframes = [c0f,c1f,c2f,c3f]
allsongs = pd.concat(fourframes)
allsongs
allsongs.to_csv('spotify.csv')

# Continue below

# If the user already did the algorithm

In [4]:
%%time
#Gets song that is currently playing (CPT)
current_song1 = sp.current_user_playing_track()

if current_song1 is None:
    current_song1 = 'spotify:track:4qMlF2uBW2PtGSEMHPcNnZ'
    # Aminé - Riri
    print('method still broken')

#gets the track URI
current_track_uri = current_song1['item']['uri']

Wall time: 176 ms


In [7]:
#Reads CSV file to DataFrame
allsongs = pd.read_csv('spotify.csv')

In [8]:
%%time
#Finds CPT in allsongs DataFrame
playing_track = allsongs.loc[allsongs['uri'] == current_track_uri]
playing_track

Wall time: 3.98 ms


,Unnamed: 0,name,artists,year,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,Cluster,uri
934,183,Empire State Of Mind,JAY-Z,2009,0.0295,0.491,0.956,0.0,11,0.46,-1.538,1,0.392,173.585,0.811,3,spotify:track:2igwFfvr1OAGX9SKDCPBwO


In [9]:
%%time
#Gets CPT cluster #
track_cluster = playing_track.iloc[0]['Cluster']

#Gets CPT year of release
playing_track_year = playing_track.iloc[0]['year']

Wall time: 1.03 ms


In [10]:
%%time
#Creates list of songs that match the CPT cluster and is within +-5 years apart
add_to_queue = allsongs[(playing_track_year >= (allsongs['year'] - 5)) & (playing_track_year <= (allsongs['year'] + 5))
        & (track_cluster == allsongs['Cluster'])].sample(10)

#Gets URI of each track and adds to list
to_queue = add_to_queue["uri"].tolist()

Wall time: 73.4 ms


In [11]:
%%time
#Adds songs to queue on Spotify
for track in to_queue:
    sp.add_to_queue(track)

Wall time: 2.49 s
